# Introdução à Banco de Dados | Trabalho Prático III
Abel Severo Rocha, Ana Carla Fernandes, Natasha Caxias
Prof. Dr. Altigran Soares da Silva

## Parte I
Verificação dos parâmetros do hardware e do software utilizado e familiarização com o ambiente do PostgreSQL.

### Tarefa 1 - Identificação do Sistema

In [1]:
import platform
import os
import shutil
import subprocess
import datetime
import psycopg2
import pandas as pd
from tabulate import tabulate
from psycopg2.extras import RealDictCursor
from tpch_pgsql import main as tpch_pgsql
from tpch4pgsql import postgresqldb as pgdb, result as r

In [2]:
def mostrarSoInfo():
    
    soInfo = platform.freedesktop_os_release()
    print("\n--------- Sistema Operacional (SO) ---------\n")
    print("Sistema Operacional ", platform.system())
    print("Versão              ", soInfo["VERSION"])
    print("Release (kernel)    ", platform.release())
    print("Arquitetura         ", platform.machine())
    print("Distribuição        ", soInfo["NAME"])


In [3]:
def mostrarHardwareInfo():

    cpuInfo = {}
    with open("/proc/cpuinfo") as f:
        for line in f:
            if ":" in line:
                k, v = line.split(":", 1)
                cpuInfo[k.strip()] = v.strip()
    
    ramInfo = {}
    with open("/proc/meminfo") as f:
        for line in f:
            k, v = line.split(":", 1)
            ramInfo[k] = v.strip()

    print("\n------------- Sobre o Hardware -------------\n")
    print("Processador:")
    print("  • Modelo          ", cpuInfo["model name"])
    print("  • CPUs lógicas    ", os.cpu_count())
    print("  • Clock base      ", cpuInfo["cpu MHz"], "MHz")

    print("\nCache:")
    with open("/sys/devices/system/cpu/cpu0/cache/index0/size") as f:
        print("  • L1d             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index1/size") as f:
        print("  • L1i             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index2/size") as f:
        print("  • L2              ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index3/size") as f:
        print("  • L3              ", f.read().strip()[:-1]+" kB")

    print("\nMemória RAM:")
    print("  • Total           ", ramInfo["MemTotal"])
    print("  • Livre           ", ramInfo["MemFree"])
    print("  • Disponível      ", ramInfo["MemAvailable"])

    total, usado, livre = shutil.disk_usage("/")
    print("\nDisco:")
    print(f"  • Total            {total/1024**3:.2f} GB")
    print(f"  • Usado            {usado/1024**3:.2f} GB")
    print(f"  • Livre            {livre/1024**3:.2f} GB")


In [4]:
mostrarSoInfo()
mostrarHardwareInfo()


--------- Sistema Operacional (SO) ---------

Sistema Operacional  Linux
Versão               24.04.3 LTS (Noble Numbat)
Release (kernel)     6.8.0-87-generic
Arquitetura          x86_64
Distribuição         Ubuntu

------------- Sobre o Hardware -------------

Processador:
  • Modelo           11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz
  • CPUs lógicas     8
  • Clock base       2861.978 MHz

Cache:
  • L1d              48 kB
  • L1i              32 kB
  • L2               1280 kB
  • L3               12288 kB

Memória RAM:
  • Total            7670432 kB
  • Livre            195052 kB
  • Disponível       1529732 kB

Disco:
  • Total            233.18 GB
  • Usado            46.06 GB
  • Livre            175.20 GB


### Tarefa 2 - Verificação de parâmetros de armazenamento

In [5]:
# TESTAR NO LAB
# sudo visudo
# seu_usuario ALL=(ALL) NOPASSWD: /sbin/hdparm

In [6]:
BASE = "/sys/block"

def run(cmd):
    return subprocess.check_output(cmd, capture_output=True, text=True).strip()

def getDispositivo():
    for dev in os.listdir(BASE):
        if dev.startswith(("loop", "ram", "dm-")): # ignora
            continue
        return dev # encontra dispositivo
    return None

def ehHD(dispositivo):
    rotational_path = os.path.join(BASE, dispositivo, "queue/rotational")
    
    if os.path.exists(rotational_path):
        with open(rotational_path) as f:
            return f.read().strip() == "1"
    return False

def mostrarSSDInfo():
    pass

def mostrarDiscoInfo():
    cmd = "sudo hdparm -I /dev/sda | grep -i geometry -A1"
    superficies = run(cmd)
    cmd = "sudo hdparm -I /dev/sda | grep -i cylinder"
    cilindros = run(cmd)
    cmd = "sudo hdparm -I /dev/sda | grep -i sectors/track"
    setores_trilha = run(cmd)
    cmd = "sudo smartctl -i /dev/sda | grep -i rotation"
    rpm = run(cmd)
    latencia = 3000/rpm
    cmd = "sudo fio --filename=/dev/sdX --direct=1 --rw=randread --bs=4k --iodepth=1 \
         --numjobs=1 --time_based --runtime=30 --name=seektest --group_reporting"
    # t_seek_avg = 
    # t_seek_max =
    # t_seek_min =
    # t_prox_trilha =
    cmd = "sudo hdparm -t /dev/sdX"
    taxa_transf = run(cmd)
    # leitura: sudo fio --filename=/dev/sdX --direct=1 --rw=read --bs=1M --iodepth=1 --time_based --runtime=20 --name=readtest --group_reporting
    # escrita: sudo fio --filename=/dev/sdX --direct=1 --rw=write --bs=1M --iodepth=1 --time_based --runtime=20 --name=writetest --group_reporting

    cmd = "stat"+getDispositivo(BASE)
    stat = run(cmd)
    
    cmd = "sudo tune2fs -l /dev/sdXn | grep 'Block size'"
    tamanho_bloco = run(cmd)

    # mudar tamanho do bloco: sudo mkfs.ext4 -b 4096 /dev/sdXn
    # notar que apaga tudo, recria o sistema de arquivos


In [7]:
disp = getDispositivo()
if ehHD(disp):
    mostrarDiscoInfo()
else:
    print("Dispositivo de armazenamento é um SSD.")

Dispositivo de armazenamento é um SSD.


### Tarefa 3 – Geração de um BD para testes


Necessário dar permissão de escrita para a pasta _tpch4pgsql_:

``chmod -R u+rwX,g+rwX,o+rX .``

#### Preparar
A fase de preparação compila o TPC-H dbgen e querygen e cria os arquivos de carga e atualização (atualização/exclusão).

In [9]:
tpch_pgsql(phase="prepare")

make: Nothing to be done for 'all'.
built dbgen from source


TPC-H Population Generator (Version 2.14.0)
Copyright Transaction Processing Performance Council 1994 - 2010
Generating data for suppliers table
Preloading text ...                                                                                                                                                                                                                                                                                                              1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

generated data for the load phase


TPC-H Population Generator (Version 2.14.0)
Copyright Transaction Processing Performance Council 1994 - 2010
Generating update pair #1 for orders/lineitem tables
Preloading text ...                                                                                                                                                                                                                                                                                                              1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

generated data for the update phase
generated data for the delete phase
created data files in ./data
created query files in ./query_root


#### Carregamento

A fase de carregamento (load) limpa o banco de dados (se necessário), carrega as tabelas no banco de dados e cria índices para consultas. Os resultados desta fase consistem nas seguintes métricas:

* Tempo de criação do esquema
* Tempo de carregamento dos dados
* Tempo de criação de restrições de chave estrangeira e índices

In [10]:
tpch_pgsql(phase="load")

dropped existing tables
cleaned database tpch
done creating schemas
done loading data to tables
done creating indexes and foreign keys
=========================== Load ===========================
create_schema: : 0:00:00.063225
load_data: 0:00:25.272934
index_tables: 0:00:26.518753
======================= End Results ========================


#### Consultas

Nessa fase, as consultas serão analizadas com o comando ``EXPLAIN ANALYZE``, evidenciando:

* Tempo de execução do planejamento;
* Tempo de execução do _exlpain_;
* Algoritmos utilizados.

Em seguida, as consultas serão executas, exibindo as seguintes informações:

* Até 10 linhas do resultado da consulta;
* Total de linhas encontradas;
* Tempo de execução.

In [11]:
class PGDBWithResults(pgdb.PGDB):

    def __init__(self, host, port, database, user, password):
        # Chamar o construtor da classe pai corretamente
        self.conn = psycopg2.connect(
                    host=host,
                    port=port,
                    dbname=database,
                    user=user,
                    password=password
                ) 
        if hasattr(self, 'conn') and self.conn:
            print("Conexão estabelecida com sucesso!")
        else:
            print("Atenção: Conexão não estabelecida")
    
    def executeQueryFromFileWithResults(self, filepath):
        """Executa query de arquivo e retorna resultados"""
        try:
            # Verificar se a conexão existe
            if not hasattr(self, 'conn') or not self.conn:
                return {'error': 'Conexão não disponível'}
            
            with open(filepath, 'r') as f:
                query = f.read()
            
            # Usar cursor que retorna dicionários
            with self.conn.cursor(cursor_factory=RealDictCursor) as cursor:
                cursor.execute(query)
                
                if cursor.description:  # Se é uma query SELECT
                    results = cursor.fetchall()
                    columns = [desc[0] for desc in cursor.description]
                    return {
                        'columns': columns,
                        'data': results,
                        'rowcount': cursor.rowcount
                    }
                else:  # Para INSERT, UPDATE, DELETE
                    return {
                        'rowcount': cursor.rowcount,
                        'message': f"Query executada: {cursor.rowcount} linhas afetadas"
                    }
                    
        except Exception as e:
            return {'error': str(e)}
        
    def executeQuery(self, query_string):
        """Executa uma query diretamente a partir de string"""
        try:
            if not self.conn:
                return {'error': 'Conexão não disponível'}
            
            with self.conn.cursor(cursor_factory=RealDictCursor) as cursor:
                cursor.execute(query_string)
                
                if cursor.description:  # Se é uma query SELECT
                    results = cursor.fetchall()
                    columns = [desc[0] for desc in cursor.description]
                    return {
                        'columns': columns,
                        'data': results,
                        'rowcount': cursor.rowcount
                    }
                else:  # Para INSERT, UPDATE, DELETE
                    self.conn.commit()
                    return {
                        'rowcount': cursor.rowcount,
                        'message': f"Query executada: {cursor.rowcount} linhas afetadas"
                    }
                    
        except Exception as e:
            return {'error': str(e)}

In [12]:
host = "localhost"
port = 5432
user = "postgres"
password = "test123"
database = "tpch"
filepath = "query_root/perf_query_gen/"

conn = PGDBWithResults(host, port, database, user, password)

# Definir algoritmos de junção e varredura comuns no PostgreSQL
algoritmos_juncao = ['Nested Loop', 'Hash Join', 'Merge Join']
algoritmos_varredura = ['Seq Scan', 'Index Scan', 'Index Only Scan', 'Bitmap Heap Scan', 'Bitmap Index Scan']
algoritmos_ordenacao = ['Sort', 'Incremental Sort']
algoritmos_agregacao = ['HashAggregate', 'GroupAggregate']

for i in [1, 3, 5, 6, 7, 9, 10, 12]:
    print(f"\n{'=' * 120}")
    print(f"EXPLAIN ANALYZE - QUERY {i}")
    print(f"{'=' * 120}")
    
    # Ler a query original
    with open(filepath + f"{i}.sql", 'r') as f:
        original_query = f.read()
    
    # Adicionar EXPLAIN ANALYZE
    explain_query = "EXPLAIN ANALYZE " + original_query
    
    start_time = datetime.datetime.now()
    result = conn.executeQuery(explain_query)
    end_time = datetime.datetime.now()
    execution_time = end_time - start_time
    
    if 'error' in result:
        print(f"Erro no EXPLAIN ANALYZE {i}: {result['error']}")
    elif 'data' in result:
        print(f"\nTempo de execução do EXPLAIN: {execution_time}")
        
        # Coletar algoritmos utilizados
        algoritmos_encontrados = {
            'juncao': [],
            'varredura': [],
            'ordenacao': [],
            'agregacao': [],
            'outros': []
        }
        
        print(f"\nPLANO DE EXECUÇÃO (EXPLAIN ANALYZE):")
        for idx, row in enumerate(result['data']):
            plan_line = list(row.values())[0] if row else ""
            print(f"{plan_line}")
            
            # Identificar algoritmos na linha do plano
            for algoritmo in algoritmos_juncao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['juncao']:
                        algoritmos_encontrados['juncao'].append(algoritmo)
            
            for algoritmo in algoritmos_varredura:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['varredura']:
                        algoritmos_encontrados['varredura'].append(algoritmo)
            
            for algoritmo in algoritmos_ordenacao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['ordenacao']:
                        algoritmos_encontrados['ordenacao'].append(algoritmo)
            
            for algoritmo in algoritmos_agregacao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['agregacao']:
                        algoritmos_encontrados['agregacao'].append(algoritmo)
        
        # Mostrar resumo dos algoritmos utilizados
        print(f"\n{'─' * 80}")
        print("ALGORITMOS IDENTIFICADOS NO PLANO DE EXECUÇÃO:")
        print(f"{'─' * 80}")
        
        if algoritmos_encontrados['juncao']:
            print(f"Algoritmos de Junção: {', '.join(algoritmos_encontrados['juncao'])}")
        
        if algoritmos_encontrados['varredura']:
            print(f"Algoritmos de Varredura: {', '.join(algoritmos_encontrados['varredura'])}")
        
        if algoritmos_encontrados['ordenacao']:
            print(f"Algoritmos de Ordenação: {', '.join(algoritmos_encontrados['ordenacao'])}")
        
        if algoritmos_encontrados['agregacao']:
            print(f"Algoritmos de Agregação: {', '.join(algoritmos_encontrados['agregacao'])}")
        
        # Verificar se não foram encontrados algoritmos conhecidos
        total_algoritmos = (len(algoritmos_encontrados['juncao']) + 
                          len(algoritmos_encontrados['varredura']) + 
                          len(algoritmos_encontrados['ordenacao']) + 
                          len(algoritmos_encontrados['agregacao']))
        
        if total_algoritmos == 0:
            print("ℹNenhum algoritmo específico identificado (possivelmente plano simples)")
            
    else:
        print(f"{result.get('message', 'Resultado inesperado')}")
        print(f"Tempo: {execution_time}")

Conexão estabelecida com sucesso!

EXPLAIN ANALYZE - QUERY 1

Tempo de execução do EXPLAIN: 0:00:03.128054

PLANO DE EXECUÇÃO (EXPLAIN ANALYZE):
Limit  (cost=231655.62..231655.94 rows=1 width=236) (actual time=3086.325..3092.493 rows=1 loops=1)
  ->  Finalize GroupAggregate  (cost=231655.62..231657.57 rows=6 width=236) (actual time=3062.960..3069.128 rows=1 loops=1)
        Group Key: l_returnflag, l_linestatus
        ->  Gather Merge  (cost=231655.62..231657.02 rows=12 width=236) (actual time=3062.908..3069.079 rows=4 loops=1)
              Workers Planned: 2
              Workers Launched: 2
              ->  Sort  (cost=230655.59..230655.61 rows=6 width=236) (actual time=3043.988..3043.989 rows=3 loops=3)
                    Sort Key: l_returnflag, l_linestatus
                    Sort Method: quicksort  Memory: 27kB
                    Worker 0:  Sort Method: quicksort  Memory: 27kB
                    Worker 1:  Sort Method: quicksort  Memory: 27kB
                    ->  Partial

In [13]:
host = "localhost"
port = 5432
user = "postgres"
password = "test123"
database = "tpch"
filepath = "query_root/perf_query_gen/"

conn = PGDBWithResults(host, port, database, user, password)

for i in [1, 3, 5, 6, 7, 9, 10, 12]:
    print(f"\n{'=' * 120}")
    print(f"QUERY {i}")
    print(f"{'=' * 120}")
    
    start_time = datetime.datetime.now()
    result = conn.executeQueryFromFileWithResults(filepath + f"{i}.sql")
    end_time = datetime.datetime.now()
    execution_time = end_time - start_time
    
    if 'error' in result:
        print(f"Erro na query {i}: {result['error']}")
    elif 'columns' in result:
        # Converter para DataFrame do pandas
        df = pd.DataFrame(result['data'], columns=result['columns'])
        
        print(f"Executada com sucesso!")
        print(f"Tempo de execução: {execution_time}")
        print(f"Total de linhas: {result['rowcount']}")
        
        if not df.empty:
            print(f"\nResultados (primeiras 10 linhas):")
            print(tabulate(df.head(10), headers='keys', tablefmt='grid', showindex=True))
        else:
            print("Nenhum resultado retornado")
            
    else:
        print(f"{result['message']}")
        print(f"Tempo: {execution_time}")

Conexão estabelecida com sucesso!

QUERY 1
Executada com sucesso!
Tempo de execução: 0:00:03.014522
Total de linhas: 1

Resultados (primeiras 10 linhas):
+----+----------------+----------------+-------------+------------------+------------------+--------------+-----------+-------------+------------+---------------+
|    | l_returnflag   | l_linestatus   |     sum_qty |   sum_base_price |   sum_disc_price |   sum_charge |   avg_qty |   avg_price |   avg_disc |   count_order |
+====+================+================+=============+==================+==================+==============+===========+=============+============+===============+
|  0 | A              | F              | 3.77341e+07 |      5.65866e+10 |      5.37583e+10 |  5.59091e+10 |    25.522 |     38273.1 |  0.0499853 |       1478493 |
+----+----------------+----------------+-------------+------------------+------------------+--------------+-----------+-------------+------------+---------------+

QUERY 3
Executada com sucesso!